In [1]:
import bagpy
from bagpy import bagreader

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
bag = bagreader("data/test1.bag")
#bag.topic_table

[INFO]  Successfully created the data folder data/test1.


In [3]:
accel_df = pd.read_csv(bag.message_by_topic('/imu/acceleration'))
accel_df.head(3)

,Time,header.seq,header.stamp.secs,header.stamp.nsecs,header.frame_id,vector.x,vector.y,vector.z
0,1.669803e+09,250,1669803320,423047544,imu_link,-1.434981,0.498791,9.674213
1,1.669803e+09,251,1669803320,428319850,imu_link,-1.426545,0.546572,9.681358
2,1.669803e+09,252,1669803320,432516182,imu_link,-1.436896,0.555246,9.700562


In [4]:
gyro_df = pd.read_csv(bag.message_by_topic('/imu/angular_velocity'))
gyro_df.head(3)

,Time,header.seq,header.stamp.secs,header.stamp.nsecs,header.frame_id,vector.x,vector.y,vector.z
0,1.669803e+09,250,1669803320,423047544,imu_link,0.003190,-0.004760,0.003020
1,1.669803e+09,251,1669803320,428319850,imu_link,-0.001221,-0.005975,-0.000975
2,1.669803e+09,252,1669803320,432516182,imu_link,0.001955,-0.001276,-0.003164


In [7]:
# get relevant data

accel_df['time_diff_ns'] = accel_df['header.stamp.secs'].diff(1) + accel_df['header.stamp.nsecs'].diff(1) / 1e9
timesteps = accel_df['time_diff_ns'].to_numpy()
vec_accel = accel_df[['vector.x', 'vector.y', 'vector.z']].to_numpy()
vec_gyro = gyro_df[['vector.x', 'vector.y', 'vector.z']].to_numpy()

In [ ]:
# do calibrations and averaging


In [ ]:
# do some integrations


In [ ]:
# plot it
fig = plt.figure()
ax = plt.axes(projection='3d')
xdata = accel_df['position_x'].values
ydata = accel_df['position_y'].values
zdata = accel_df['position_z'].values

ax.scatter(xdata, ydata, zdata)
plt.show()